In [1]:
import json

In [3]:
with open('train_graph.json', 'r') as f:
    train_data_full = json.load(f)
with open('test_graph.json', 'r') as f:
    test_data = json.load(f)

In [4]:
# merge data
train_data = {node['id']: node for node in train_data_full.pop('nodes')}
test_data = {node['id']: node for node in test_data['nodes']}
merged_data = {}

print(len(train_data))
print(len(test_data))

123
123


In [5]:
infoNodeCount = sum(map(lambda node_key: train_data[node_key]['type'] == 'infoNode', train_data))
print("Info nodes", infoNodeCount)
print("Response Nodes", sum(map(lambda node_key: test_data[node_key]['type'] == 'userResponseNode', test_data)))
print("Train FAQ questions", sum(map(lambda node_key: len(train_data[node_key]['data']['questions']), train_data)))
print("Train FAQ questions per info node", sum(map(lambda node_key: len(train_data[node_key]['data']['questions']), train_data)) / infoNodeCount)
print("Test FAQ questions", sum(map(lambda node_key: len(test_data[node_key]['data']['questions']), test_data)))
print("Test FAQ questions per info node", sum(map(lambda node_key: len(test_data[node_key]['data']['questions']), test_data)) / infoNodeCount)

Info nodes 79
Response Nodes 23
Train FAQ questions 279
Train FAQ questions per info node 3.5316455696202533
Test FAQ questions 173
Test FAQ questions per info node 2.189873417721519


In [6]:
for key in train_data:
    assert key in test_data

    merged_data[key] = train_data[key]
    train_keys = set(q['id'] for q in train_data[key]['data']['questions'])
    test_keys = set(map(lambda q: str(int(q['id'])+2), test_data[key]['data']['questions']))
    assert  train_keys.isdisjoint(test_keys), f"Train: {train_keys}, Test: {test_keys}, Intersection: {train_keys.intersection(test_keys)}"
    for q in test_data[key]['data']['questions']:
        q['id'] = str(int(q['id'])+2)
    merged_data[key]['data']['questions'] += test_data[key]['data']['questions']

assert len(merged_data) == len(train_data) == len(test_data)


In [7]:
final_data = {
    'nodes': list(merged_data.values()),
} | train_data_full

In [8]:
infoNodeCount = sum(map(lambda node_key: merged_data[node_key]['type'] == 'infoNode', merged_data))
print("Info nodes", infoNodeCount)
print("Response Nodes", sum(map(lambda node_key: merged_data[node_key]['type'] == 'userResponseNode', merged_data)))
print("Merged FAQ questions", sum(map(lambda node_key: len(merged_data[node_key]['data']['questions']), merged_data)))
print("merged_data FAQ questions per info node", sum(map(lambda node_key: len(merged_data[node_key]['data']['questions']), merged_data)) / infoNodeCount)

Info nodes 79
Response Nodes 23
Merged FAQ questions 452
merged_data FAQ questions per info node 5.7215189873417724


In [6]:
with open('traintest_graph.json', 'w') as f:
    json.dump(final_data, f)

In [9]:
# merge answer synonyms
with open('train_answers.json', "r") as f:
    train_answers = json.load(f)
with open('test_answers.json', "r") as f:
    test_answers = json.load(f)

In [10]:
from statistics import mean

print("Train Answers", sum(map(lambda ans: len(train_answers[ans]), train_answers)))
print("Train Answers synonyms per answer", mean(map(lambda ans: len(train_answers[ans]), train_answers)))
print("Test Answers", sum(map(lambda ans: len(test_answers[ans]), test_answers)))
print("Test Answers synonyms per answer", mean(map(lambda ans: len(test_answers[ans]), test_answers)))

Train Answers 246
Train Answers synonyms per answer 3.3698630136986303
Test Answers 162
Test Answers synonyms per answer 2.219178082191781


In [11]:
merged_answers = {}
for key in train_answers:
    assert key in test_answers
    merged_answers[key] = train_answers[key] + test_answers[key]

In [12]:
assert len(merged_answers) == len(train_answers) == len(test_answers)

In [13]:
from statistics import mean

print("Merged Answers", sum(map(lambda ans: len(merged_answers[ans]), merged_answers)))
print("Merged Answers synonyms per answer", mean(map(lambda ans: len(merged_answers[ans]), merged_answers)))

Merged Answers 408
Merged Answers synonyms per answer 5.589041095890411


In [11]:
with open('traintest_answers.json', "w") as f:
    json.dump(merged_answers, f)